# Exercice: Modélisation orientée requête (Cassandra)

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/cassandra.png" width="100" height="100"></td>
    </tr></table>



### Dans ce notebook, nous allons travailler sur la modélisation orientée requête sous la BDD No-SQL Cassandra, en particulier nous traitons les points suivants :

* Dénormaliser une BDD relationnelle en partant du principe 1 table per query 
* Définir la partie Primary Key pour chaque requête
* Optimiser vos modèles en choisissant la bonne Partition Key & Clustering Column
* Valider vos modèles 
* Tester quelques requêtes particulières


#### installer cassandra-driver afin de se connecter au cluster cassandra
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [ ]:
import cassandra

### créer une connection au cluster cassandra

In [ ]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) # si cassandra est installé localement avec le port par défaut 9042
    session = cluster.connect()
except Exception as e:
    print(e)

### créer maintenant un keyspace nommé "iot_esi"

In [ ]:
try:
    
  ### add code here 

except Exception as e:
    print(e)

### connecter à votre Keyspace  iot_esi 


In [ ]:
try:
    
    ### add code here 
    
except Exception as e:
    print(e)

### Supposons qu'on a un système IoT qui stocke les données capturées par les différents Sensors en utilisant un modèle RELATIONNEL, tel que est décrit dans le schéma ER ci-dessous:

<table><tr>
    <td><img src="image/ex1_IoT_ER_model.png" width="500" height="500"></td>
      </tr></table>

### imaginons  que nous souhaitons analyser ces données à  travers 3 requêtes analytiques:
* `Query1= retourner les informations(name, region, nbSensors) de tous les Networks`
* `Query2= retourner les informations (name,type,localisation) de tous les Sensors d'un Network donné.`
* `Query3= retourner le AVG_measurements par sensor,  par jour et par heure.  Ordonner le résultat par  jour(desc) et par heure (asc)`

#### Créer les trois tables qui correspondent aux requêtes précédentes, en indiquant pour chacune:
* Primary Key
* Partition Key
* Clustering Column

In [ ]:
try:
    
    ### Create the table that corresponds to the Query1
    query1="...."
    session.execute(query1)

    ### Create the table that corresponds to the Query2
    query2="...."
    session.execute(query2)
    
    
    ### Create the table that corresponds to the Query3
    query3="...."
    session.execute(query3)
    
    
except Exception :e
        print(e)

### Une fois vous créez les différentes tables, essayez de les peupler en se basant sur le contenu de la BDD relationnelle.

<table><tr>
    <td><img src="image/network_data.png" width="200" height="300"></td></tr>
   <tr> <td><img src="image/sensor_data.png" width="400" height="500"></td> </tr>
        <tr>
       <td><img src="image/recorded_data.png" width="350" height="250"></td>
      </tr>
</table>

### Insertion des données dans Table1

In [ ]:
query = "INSERT INTO ... (......)"
query = query + " VALUES (%s, %s, %s, ...)"

try:
    session.execute(query, (.......))
    session.execute(query, (.......))
except Exception :e
        print(e)    

### Insertion des données dans Table2

In [ ]:
query = "INSERT INTO ... (......)"
query = query + " VALUES (%s, %s, %s, ...)"

try:
    session.execute(query, (.......))
    session.execute(query, (.......))
except Exception :e
        print(e)    

### Insertion des données dans Table3

In [ ]:
query = "INSERT INTO ... (......)"
query = query + " VALUES (%s, %s, %s, ...)"

try:
    session.execute(query, (.......))
    session.execute(query, (.......))
except Exception :e
        print(e)    

### Valider votre modèle en exécutant les requêtes suivantes

* Retourner le nombre de sensors dans la région ouest"

* Retourner les sensors qui sont placés dans le network "esi-net"

* Retourner la température maximale enregistrée par le sensor "temp-454" le 31-10-2020

In [ ]:
# query1
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

In [ ]:
# query2
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

In [ ]:
# query3
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

### Essayons d'autres requêtes 

* `Retourner la température maximale enregistrée par le sensor "temp-454" à 13:00:00 pendant le  mois d'octobre` 

In [ ]:
# query4
query = "...."


try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)      

### Proposer une solution pour optimiser cette requête

In [ ]:
### make your solution 


###  drop  tables. 

In [ ]:
query = "....."
rows = session.execute(query)

query = "....."
rows = session.execute(query)

query = "....."
rows = session.execute(query)


### close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()